This notebook provides comparative analysis of the regression models retrived according the three approaches, namely parametric, semi-parametric, and non-parametric regression models. 

In [ ]:
# Modules
from termcolor import colored
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
import statistics

import os
from os import listdir
from os.path import isfile, join

import pickle

from pathlib import Path
import sys
PROJECT_DIR =Path(os.path.abspath('')).parents[1]
sys.path.append(os.fspath(PROJECT_DIR))

from pipeline.regression_fx import create_model, realElbows
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import random

import matplotlib as mpl
import colorsys
import matplotlib.colors as mc
import itertools

from pipeline.definitions import *
from localreg import *

## Define general parameters for figures

In [ ]:
graph_setting="notebook" #or "article"

In [ ]:
if graph_setting=="article":
    
    #journal-quality parameter settings
    resolution_factor=2
    desired_font=10

elif graph_setting=="notebook":
    resolution_factor=1
    desired_font=12

#conversion factors
cm_to_inch=0.393701
classic_proportion=6.4/4.8
golden_rate=1.618

#Elsevier column width is 8.4 cm, double-column width is 17.7 cm (in inches: 3.31 and 6.97)
small_figsize=(resolution_factor*3.31, resolution_factor*3.31/classic_proportion)
big_figsize=(resolution_factor*6.97, resolution_factor*6.97/classic_proportion)
square_fig=(resolution_factor*4.5, resolution_factor*4.5)

#define colors palette
colors={}
colors["P_phys"]="sandybrown" 
colors["P_semiPar"]="indianred"
colors["P_blackbox"]="mediumpurple"

#changings regarding fonttypex
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = "Arial"

desired_font=10
font_size=resolution_factor*desired_font

#define path for figures
figures_path=FIGURES
#check existance of figure path
if not os.path.exists(figures_path):
    print("The selected directory to store figures does not exist")

## Data Import

In [ ]:
#See available case studies
dataset_path=DATA_NORMALIZED
onlyfiles = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]
print('The following datasets are available')
print(onlyfiles)

In [ ]:
# Import a case study dataset using pandas
method_short="ts" #that is, time serie, or ew, that is elementWise
norm_dataset = pd.read_csv(os.path.join(DATA_NORMALIZED, 'norm_'+method_short+'_buildings_dataset.csv'))

#load dataset with original measures
dataset_name='buildings_dataset_clean.csv'
dataset = pd.read_csv(DATA_CLEAN+"/"+dataset_name)

print("Shape of dataset: "+str(dataset.shape))
dataset.tail()
#load dataset description
data_info=pd.read_csv(DATASETS+"/"+'buildings_data_description.csv')

## Models Import

In [ ]:
#create the list of IDs for the case studies, a list for the models and proper matrices to store performance results
IDs=norm_dataset.ID.unique()
models_name_list=["Parametric", "Semi-Parametric", "Non-Parametric"]
models_list=['P_phys', 'P_semiPar', 'P_blackbox']
df_r2=pd.DataFrame(index=pd.MultiIndex.from_product([models_name_list, ['Train', 'Test']]), columns=IDs)
df_mape=df_r2.copy()
df_mae=df_r2.copy()

In [ ]:
#Load all the models into a dictionary
models={}

for caseStudy in IDs:

    #check names of the different models for the selected case study
    ES_models_list=[f for f in listdir(PAR_MODELS) if isfile(join(PAR_MODELS, f)) and caseStudy in f]
    semiPar_submodel_list=[f for f in listdir(SEMIPAR_MODELS) if isfile(join(SEMIPAR_MODELS, f)) and caseStudy in f]
    blackbox_models_list=[f for f in listdir(BLACKBOX_MODELS) if isfile(join(BLACKBOX_MODELS, f)) and caseStudy in f]
    
    #ensure 1 model and no more is available for each case study and model type
    if (len(ES_models_list)>1) | (len(semiPar_submodel_list)>1) | (len(blackbox_models_list)>1):
        print(colored("WARNING: Multiple models have been found for case study "+caseStudy+"\nThe first model in alphabetical order will be loaded", "yellow"))
    
    if (len(ES_models_list)==0) | (len(semiPar_submodel_list)==0) | (len(blackbox_models_list)==0) :
        print(colored("ERROR: One or more of the regression model is missing for case study "+caseStudy+"\nThe following models exist for the selected case study: ", "red"))
        print(pod_models_list)
    else: 
        ES_modelname=ES_models_list[0]
        semiPar_modelname=semiPar_submodel_list[0]
        blackbox_modelname=blackbox_models_list[0]

    #load models
    ES_model=tf.keras.models.load_model(PAR_MODELS+'/'+ES_modelname, custom_objects={'realElbows':realElbows})
    semiPar_model=tf.keras.models.load_model(SEMIPAR_MODELS+'/'+semiPar_modelname, custom_objects={'realElbows':realElbows})
    blackbox_model=tf.keras.models.load_model(BLACKBOX_MODELS+'/'+blackbox_modelname)
    models[caseStudy, "ES"]=ES_model
    models[caseStudy, "semiPar"]=semiPar_model
    models[caseStudy, "BlackBox"]=blackbox_model
    #model.summary()

print(colored(str(len(models))+" models have been successfully loaded", "green"))

## Models prediction and performance

In [ ]:
#select input and output variables
ind_var=["T_a", "RH", "Wv", "atmP", "G", "s_Wa", "c_Wa", "s_H", "c_H", "s_D", "c_D", "dayType"]
modeled_ind_var="T_a"
dep_var="P"

#create new columns in dataset
dataset["P_phys"]=np.nan
dataset["P_res"]=np.nan
dataset["P_semiPar"]=np.nan
dataset["P_blackbox"]=np.nan

#load scalers
directory=DATA
file_name="scalers_"+method_short
with open(directory+"/"+file_name+".pkl", 'rb') as f:
    scalers_dict = pickle.load(f)

In [ ]:
for caseStudy in IDs:
  
    #define modeled input
    df=norm_dataset.loc[norm_dataset.ID==caseStudy]
    df_rescaled=dataset.loc[norm_dataset.ID==caseStudy]
    x=pd.DataFrame(df[ind_var])
    x_modeled=pd.DataFrame(df[modeled_ind_var])
    y=pd.DataFrame(df[dep_var])
    
    #split test and train 
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
    x_modeled_train=pd.DataFrame(x_train[modeled_ind_var])
    x_modeled_test=pd.DataFrame(x_test[modeled_ind_var])
    
    pd.options.mode.chained_assignment = None  # default='warn', use it to hide the SettingCopyWarning
    
    #predict parametric model output
    ES_model=models[caseStudy, "ES"]                                              #load model
    #x_modeled_NN=scalers_dict[caseStudy, "x_modeled"].transform(x_modeled)         #scale input data
    #y_NN_pred_ES=ES_model.predict(x_modeled_NN)                                   #predict output
    #df["P_phys"]=scalers_dict[caseStudy, "y"].inverse_transform(y_NN_pred_ES)     #rescale output
    df["P_phys"]=ES_model.predict(x_modeled)  
    
    #calculate parametric model residuals
    df["P_res"]=df[dep_var]-df["P_phys"]
    
    #predict semi-parametric submodel output
    semiPar_model=models[caseStudy, "semiPar"]                                   #load model
    #x_NN=scalers_dict[caseStudy, "x"].transform(x)                                 #scale input data
    #y_NN_pred_semiPar=semiPar_model.predict((x_modeled, x), verbose=0) #y_res_NN_pred=nonPar_submodel.predict(x_NN)                                   #predict output
    #df["P_semiPar"]=y_res_pred=scalers_dict[caseStudy, "y_res"].inverse_transform(y_NN_pred_semiPar)  #rescale output
    df["P_semiPar"]=semiPar_model.predict((x_modeled, x), verbose=0)
    

    #Predict P from black box model
    blackbox_model=models[caseStudy, "BlackBox"]                                   #load model
    #y_NN_pred_bb=blackbox_model.predict(x)                                      #predict output
    #df["P_blackbox"]=scalers_dict[caseStudy, "y"].inverse_transform(y_NN_pred_bb)  #rescale output
    df["P_blackbox"]=blackbox_model.predict(x)  

    #store new results
    norm_dataset.loc[dataset.ID==caseStudy, "P_phys"]=df["P_phys"]
    norm_dataset.loc[dataset.ID==caseStudy, "P_res"]=df["P_res"]
    norm_dataset.loc[dataset.ID==caseStudy, "P_semiPar"]=df["P_semiPar"]
    norm_dataset.loc[dataset.ID==caseStudy, "P_blackbox"]=df["P_blackbox"]

    #rescale models' outputs results
    df_rescaled["P_phys"]=scalers_dict[caseStudy, "y"].inverse_transform(df["P_phys"].values.reshape(-1,1))
    df_rescaled["P_res"]=scalers_dict[caseStudy, "y"].inverse_transform(df["P_res"].values.reshape(-1,1))
    df_rescaled["P_semiPar"]=scalers_dict[caseStudy, "y"].inverse_transform(df["P_semiPar"].values.reshape(-1,1))
    df_rescaled["P_blackbox"]=scalers_dict[caseStudy, "y"].inverse_transform(df["P_blackbox"].values.reshape(-1,1))

    #store rescaled results
    dataset.loc[dataset.ID==caseStudy, "P_phys"]=df_rescaled["P_phys"]
    dataset.loc[dataset.ID==caseStudy, "P_res"]=df_rescaled["P_res"]
    dataset.loc[dataset.ID==caseStudy, "P_semiPar"]=df_rescaled["P_semiPar"]
    dataset.loc[dataset.ID==caseStudy, "P_blackbox"]=df_rescaled["P_blackbox"]
    
    #for each model calculate the performance achieved on the test set
    for model_name, output in zip(models_name_list, [df_rescaled["P_phys"], df_rescaled["P_semiPar"], df_rescaled["P_blackbox"]]):
        x=df_rescaled["T_a"]
        y_true=df_rescaled["P"]
        _, _, y_true_train, y_true_test = train_test_split(x, y_true, test_size=0.33, random_state=42)
        y_pred=output
        _, _, y_pred_train, y_pred_test = train_test_split(x, y_pred, test_size=0.33, random_state=42)
        #store regression models performance
        df_r2.loc[model_name, 'Train'][caseStudy]=r2_score(y_true_train, y_pred_train)
        df_r2.loc[model_name, 'Test'][caseStudy] =r2_score(y_true_test, y_pred_test)
        
        df_mape.loc[model_name, 'Train'][caseStudy]=mean_absolute_percentage_error(y_true_train, y_pred_train)
        df_mape.loc[model_name, 'Test'][caseStudy] =mean_absolute_percentage_error(y_true_test, y_pred_test)
        
        df_mae.loc[model_name, 'Train'][caseStudy]=mean_absolute_error(y_true_train, y_pred_train)
        df_mae.loc[model_name, 'Test'][caseStudy] =mean_absolute_error(y_true_test, y_pred_test)

print(colored("Predictions from the regression models correctly retrieved", "green"))

## Show summary of performance

In [ ]:
#calculate mean of performance results
res_summary=pd.DataFrame(index=df_r2.index, columns=["R2", "MAE", "MAPE"])
for model, model_name in zip(models_list, models_name_list):
    res_summary["R2"]=df_r2.mean(axis=1)
    res_summary["MAPE"]=df_mape.mean(axis=1)
    res_summary["MAE"]=df_mae.mean(axis=1)
#calculate std of performance results 
res_std=pd.DataFrame(index=df_r2.index, columns=["R2", "MAE", "MAPE"])
for model, model_name in zip(models_list, models_name_list):
    res_std["R2"]=df_r2.std(axis=1)
    res_std["MAPE"]=df_mape.std(axis=1)
    res_std["MAE"]=df_mae.std(axis=1)

In [ ]:
pd.options.display.float_format = "{:,.3}".format
print("Mean values accross the different case studies")
display(res_summary)

print("Standard deviation the different case studies")
display(res_std)

## Compare predictions from the different models

In [ ]:
#select cases studies
caseStudy_B=IDs[0]
caseStudy_C=IDs[1]
caseStudy_list=[caseStudy_B, caseStudy_C]

#initializa graphical obnject
fig, (ax1, ax2) = plt.subplots(2, 3, figsize=big_figsize)
subplot_pos_list=[231, 232, 233, 234, 235, 236]

#define title and subtitles
subtitle={}
subtitle[models_list[0]], subtitle[models_list[1]], subtitle[models_list[2]]=models_name_list

for case, subplot_pos in zip(list(itertools.product(caseStudy_list, models_list)), subplot_pos_list): #this cycle iterate over 6 combinations of modeltype*casestudy
    caseStudy=case[0]
    model=case[1]
    model_name=subtitle[model]
    
    #define modeled input
    df=dataset.loc[dataset.ID==caseStudy]
    scaler_y=scalers_dict[caseStudy, "y"]
    y_pred=scaler_y.transform(pd.DataFrame(df[model].values, columns=["P"]))
    y_real=scaler_y.transform(pd.DataFrame(df["P"]))

    #get limit y value
    p1 = max(max(y_pred), max(y_real))
    p2 = min(min(y_pred), min(y_real))

    #access to specific subplot
    ax=plt.subplot(subplot_pos)

    #scatter predictions
    ax.scatter(y_real, y_pred, s=1.5, color=colors[model])
    ax.plot([p1, p2], [p1, p2], color="black", linestyle='--')

    #graphical custom options
    ax.grid(True, which='both',axis='both',alpha=0.5, linewidth=0.45)
    ax.tick_params(labelsize=font_size-2)
    ticks=[0, 0.25, 0.5, 0.75, 1]
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)

    #arrange image details (ticks, tickslabels and titles)
    if subplot_pos!=234:
        if subplot_pos not in [235, 236]:
            ax.tick_params(axis='x',which='both',length=0.1,width=0.1,pad=1)
            ax.set_xticklabels([])
        if subplot_pos!=231:
            ax.tick_params(axis='y',which='both',length=0.1,width=0.1,pad=1)
            ax.set_yticklabels([])
    if subplot_pos in [231, 232, 233]:  
        ax.set_title(model_name, fontsize=font_size)
    if subplot_pos==231:
        ax.set_ylabel("a)", fontsize=font_size-2, rotation=0)
        ax.yaxis.set_label_coords(-0.25, +0.5)
    elif subplot_pos==234:
        ax.set_ylabel("b)", fontsize=font_size-2, rotation=0)
        ax.yaxis.set_label_coords(-0.25, +0.5)

    #print performance for the selected case study and model
    print("\nCase study: "+caseStudy)
    print("Model: "+model_name+ "\nMAPE:"+str(df_mape.loc[model_name, 'Test'][caseStudy]))
    print("R2:"+str(df_r2.loc[model_name, 'Test'][caseStudy]))
    


text_x=fig.supxlabel('True Values [-]', y=0.02, fontsize=font_size)
text_y=fig.supylabel('Predictions [-]', x=0.02, fontsize=font_size)
plt.tight_layout()
plt.subplots_adjust(wspace=0.03*resolution_factor, hspace=0.03*resolution_factor)
plt.show()
#fig.savefig(figures_path+"/models_forecast_comparison_2_buildings", bbox_extra_artists=(text_x, text_y), bbox_inches='tight', dpi=450)

## Correlation Analysis

In [ ]:
#select a case Study
caseStudy=IDs[1]
print(caseStudy)
df=dataset.loc[dataset.ID==caseStudy]
df_corr=df.drop(["ID", "P_res", "P_phys", "P_semiPar", "P_blackbox"], axis=1).corr()

#change order of the variables to be visualized in the heatmap
df_corr=df_corr.loc[["P", 'T_a', 'T_b', 'DP', 'RH', 'Wv', 'Wgv', 'atmP', 'G', 's_Wa', 'c_Wa',
       's_H', 'c_H', 'dayType', 's_D', 'c_D'], ["P", 'T_a', 'T_b', 'DP', 'RH', 'Wv', 'Wgv', 'atmP', 'G', 's_Wa', 'c_Wa',
       's_H', 'c_H', 'dayType', 's_D', 'c_D']]

#graphical representation of predictions accuracy
grid_spec = {"width_ratios": (.98, .04), "wspace": 0.15}

fig, (ax, cbar_ax) = plt.subplots(1,2, figsize=square_fig,   gridspec_kw=grid_spec)
#create heat map
ax=sns.heatmap(df_corr, vmin=0.0, vmax=1.0, ax=ax, square=True, cbar_ax=cbar_ax, cbar_kws={'label': 'Pearson Correlation Coefficient [-]'})
cbar_ax.yaxis.label.set_size(font_size)
cbar_ax.tick_params(labelsize=font_size)
ax.tick_params(labelsize=font_size)
#ax.set_title("Correlation Matrix", fontsize=font_size)

#fig.savefig(figures_path+"/full_correlation_matrix", bbox_inches='tight', dpi=450)

In [ ]:
#select reduced features for correlation analysis of power and power residuals
df_corr=df.drop(["ID"], axis=1).corr()
reduced_df_corr=df_corr.loc[["P", "P_phys", "P_res"], ["P", "P_phys", "P_res", 'T_a', 'T_b', 'DP', 'RH', 'Wv', 'Wgv', 'atmP', 'G', 's_Wa', 'c_Wa',
       's_H', 'c_H', 'dayType', 's_D', 'c_D']]


fig, ax= plt.subplots(1, figsize=square_fig)
#create heat map
ax=sns.heatmap(reduced_df_corr, vmin=0.0, vmax=1.0, ax=ax, square=True, cbar=False)
ax.tick_params(labelsize=font_size)
#ax.set_title("Correlation Matrix", fontsize=font_size)

#fig.savefig(figures_path+"/reduced_correlation_matrix",  bbox_inches='tight', dpi=450)

## Semi-parametric VS Parametric model

In [ ]:
#define general parameters for plot
show_title="No"
show_labels="No"
show_ticks="Partial"
limit_x_axis = None
limit_y_axis = None

fig, ax = plt.subplots(1, figsize=big_figsize, layout='constrained')

#select case study
caseStudy=IDs[0]
print(caseStudy)
df=dataset.loc[dataset.ID==caseStudy]

#select inputs and outputs from the parametric and semiparametric model
x=df[ind_var]
y=df["P"]
y_pred=df["P_semiPar"]
y_ref_line=df["P_phys"]
explanatory_var="T_a"

#re-order dataset according to Temperature
ordered_df=df.copy()
ordered_df['y_pred']=y_pred.values    
ordered_df=ordered_df.sort_values(by=explanatory_var)

#plot models outputs
points=[]    #create a list to store graphical elements to later show the legend
#plot real points
real_points=ax.scatter(df[explanatory_var], df[dep_var], color="gray", alpha=0.6, s=1.5*resolution_factor)
points.append(real_points)
#plot points predicted by the semi-parametric model
pred_points=ax.scatter(ordered_df[explanatory_var], ordered_df["y_pred"], color=colors['P_semiPar'], marker='x', s=1.5*resolution_factor, alpha=0.8)
points.append(pred_points)
#plot the line predicted by the semi-parametric model
ref_line=ordered_df["P_phys"].values
line,=ax.plot(ordered_df[explanatory_var], ref_line, color=colors["P_phys"], zorder=3, linewidth=2*resolution_factor) #zorder is used to define the order elements are plot
points.append(line)

#graphical custom options
ax.grid(True, which='both',axis='both',alpha=0.5, linewidth=0.45)
ax.tick_params(labelsize=font_size)
ax.set_xlabel('Outdoor Temperature [°C]', size=font_size)
ax.set_ylabel('Normalized\n Electrical Load [-]', size=font_size)
legend=fig.legend(points, ["Real Values", "SPM predictions", "PM predictions"], ncol=1, labelspacing=0., bbox_to_anchor=(0.98, 0.35), fontsize=font_size)

#fig.savefig(figures_path+"/ES_style_model_forecast",  bbox_inches='tight', dpi=450)

## Testing the impact of Temperature

The following figures will compare the regression line of parametric models and a projection of the predictions of the semiparametric model to estimate the cualitative impact of temperature for this model. The projections are obtained by using a smoothing function, by means of a local regression. 

The goal of this comparative analysis is to detect possible deviations of the complex behavior, with respect to temperature, modelled by the semi parametric model, from the one that was previously modeled by means of the parametric model (Energy Signature model).

In [ ]:
#define general parameters for plot
show_title="No"
show_labels="No"
show_ticks="Partial"
limit_x_axis = None
limit_y_axis = None



#select case studies
caseStudies=[IDs[0], IDs[1]]

fig, axs = plt.subplots(len(caseStudies), 1, figsize=(big_figsize[0], big_figsize[1]*len(caseStudies)), layout='constrained')

for caseStudy, ax in zip(caseStudies, axs.flatten()):
    
    #select inputs and outputs from the parametric and semiparametric model
    df=dataset.loc[dataset.ID==caseStudy]
    x=df[ind_var]
    y=df["P"]
    y_pred=df["P_semiPar"]
    y_ref_line=df["P_phys"]
    explanatory_var="T_a"

    #re-order dataset according to Temperature
    ordered_df=df.copy()
    ordered_df['y_pred']=y_pred.values    
    ordered_df=ordered_df.sort_values(by=explanatory_var)
    
    points=[]      #create a list to store graphical elements to later show the legend
    #plot real points
    real_points=ax.scatter(df[explanatory_var], df[dep_var], color="gray", alpha=0.6, s=1.5*resolution_factor)
    points.append(real_points)

    #get reference line of parametric model
    ref_line_Par=ordered_df["P_phys"].values
    
    #define reference line of semi-parametric model by smoothing the prediction points
    ref_line_semiPar=localreg(ordered_df[explanatory_var].values, ordered_df['P_semiPar'].values, degree=1, kernel=rbf.epanechnikov, frac=0.15)
    
    #plot lines
    line_par,=ax.plot(ordered_df[explanatory_var], ref_line_Par, color=colors["P_phys"], zorder=3, linewidth=1.8*resolution_factor) #zorder is used to define the order elements are plot
    line_semiPar,=ax.plot(ordered_df[explanatory_var], ref_line_semiPar, color=colors["P_semiPar"], zorder=3, linewidth=1.8*resolution_factor) #zorder is used to define the order elements are plot
    points.extend([line_par, line_semiPar])
    
    #graphical custom options
    ax.grid(True, which='both',axis='both',alpha=0.5, linewidth=0.45)
    ax.tick_params(labelsize=font_size)
    ax.set_xlabel('Outdoor Temperature [°C]', size=font_size)
    ax.set_ylabel('Normalized\n Electrical Load [-]', size=font_size)
    legend=fig.legend(points, ["Real Values", "PM predictions", "SPM predictions", "NPM predictions"], ncol=1, labelspacing=0., bbox_to_anchor=(0.85, 0.97), fontsize=font_size)
    #plt.tight_layout()
    
#fig.savefig(figures_path+"/deviation_temperature_impact_2_buildings", bbox_inches='tight', dpi=450)